# Rasheed Hameed
# Assignment 3 - Automatic Document Clustering


__For this problem you will use a different subset of the 20 Newsgroup data set that you used in Assignment 2  (see the description of the full dataset). The subset for this assignment includes 2,500 documents (newsgroup posts), each belonging to one of 5 categories windows (0), crypt (1), christian (2), hockey (3), forsale (4). The documents are represented by 9328 terms (stems). The dictionary (vocabulary) for the data set is given in the file "terms.txt" and the full term-by-document matrix is given in "matrix.txt" (comma separated values). The actual category labels for the documents are provided in the file "classes.txt". Your goal in this assignment is to perform clustering on the documents and compare the clusters to the actual categories. Your tasks in this problem are the following [Note: for the clustering part of this assignment you should use the kMeans module form Ch. 10 of MLA (use the version provided here as it includes some corrections to the book version). You may also use Pandas and other modules from scikit-learn that you may need for preprocessing or evaluation.]__

In [1]:
#Loading libraries

import pandas as pd
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
#to suppresss the orange warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Read the data into panda's dataframe
classes = np.genfromtxt("newsgroups5/classes.txt",skip_header=1, usecols=(1),dtype=str)
print(classes.shape)

(2500,)


In [3]:
matrix = pd.read_csv("newsgroups5/matrix.txt", sep=',',header=None)
matrix.shape

(9328, 2500)

In [4]:
terms = pd.read_csv("newsgroups5/terms.txt", sep=',', header=None,  dtype='str' )
terms=terms.iloc[:,0]
print(terms.shape)
terms.head(5)

(9328,)


0        aa
1     aargh
2     aaron
3    aaronc
4        ab
Name: 0, dtype: object

__a. Create your own distance function that, instead of using Euclidean distance, uses Cosine similarity. This is the distance function you will use to pass to the kMeans function.__

In [5]:
import random
import math


def distCosine(vecA,vecB):
    normA = linalg.norm(vecA)
    normB = linalg.norm(vecB)
    sims = dot(vecA,vecB)/(normA * normB)
    dists = 1 - sims
    return dists

def randCent(dataSet, k):
    n = dataSet.shape[1]
    centroids = np.zeros((k,n), dtype= float)
    for j in range(n): #create random cluster centers
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * np.random.rand(k)
    return centroids 

def kMeans(dataSet, k, distMeas=distCosine, createCent=randCent):
    m = dataSet.shape[0]
    clusterAssment = np.zeros((m,2), dtype=float)#create mat to assign data points 
                                      #to a centroid, also holds SE of each point
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):#for each data point assign it to the closest centroid
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        # print centroids
        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[nonzero(clusterAssment[:,0]==cent)[0]] #get all the point in this cluster - Note: this was incorrect in the original distribution.
            
            
            if(len(ptsInClust)!=0):
                centroids[cent,:] = mean(ptsInClust, axis=0) #assign centroid to mean - Note condition was added 10/28/2013
    return centroids, clusterAssment

__b. Load the data set [Note: the data matrix provided has terms as rows and documents as columns. Since you will be clustering documents, you'll need to take the transpose of this matrix so that your main data matrix is a document x term matrix. In Numpy, you may use the ".T" operation to obtain the transpose.] Then, split the data set (the document x term matrix) and set aside 20% for later use (see below). Use the 80% segment for clustering in the next part. The 20% portion must be a random subset.__

In [6]:
DT = matrix
DT.shape

(9328, 2500)

In [7]:
TD = DT.T
TD.shape

(2500, 9328)

In [8]:
NDocs = TD.shape[0]
print(NDocs)

2500


In [9]:
numTerms=TD.shape[1]
print(numTerms)

9328


In [10]:
# Find doucment frequencies for each term
DF = np.array([(TD!=0).sum(0)])
print (DF)

[[10  6 22 ...  3  3  4]]


In [11]:
NMatrix=np.ones(np.shape(TD), dtype=float)*NDocs
print (NMatrix)

[[2500. 2500. 2500. ... 2500. 2500. 2500.]
 [2500. 2500. 2500. ... 2500. 2500. 2500.]
 [2500. 2500. 2500. ... 2500. 2500. 2500.]
 ...
 [2500. 2500. 2500. ... 2500. 2500. 2500.]
 [2500. 2500. 2500. ... 2500. 2500. 2500.]
 [2500. 2500. 2500. ... 2500. 2500. 2500.]]


In [12]:
IDF = np.log2(np.divide(NMatrix, DF))
print (IDF)

[[7.96578428 8.70274988 6.82828076 ... 9.70274988 9.70274988 9.28771238]
 [7.96578428 8.70274988 6.82828076 ... 9.70274988 9.70274988 9.28771238]
 [7.96578428 8.70274988 6.82828076 ... 9.70274988 9.70274988 9.28771238]
 ...
 [7.96578428 8.70274988 6.82828076 ... 9.70274988 9.70274988 9.28771238]
 [7.96578428 8.70274988 6.82828076 ... 9.70274988 9.70274988 9.28771238]
 [7.96578428 8.70274988 6.82828076 ... 9.70274988 9.70274988 9.28771238]]


In [13]:
#compute the TFxIDF values for each document-term entry
TD_tfidf= TD * IDF

In [14]:
from sklearn.model_selection import train_test_split
train, test, target_train, target_test = train_test_split(TD_tfidf, classes, test_size=0.2, random_state=33)

In [15]:
#The other numpy conflicts with this numpy below to make the kmeans function work
from numpy import *

In [16]:
print(train.shape)
print(test.shape)
print(target_train.shape)
print(target_test.shape)

(2000, 9328)
(500, 9328)
(2000,)
(500,)


__c. Perform Kmeans clustering on the training data. Write a function to display the top N terms in each cluster along with the cluster DF values for each term and the size of the cluster. The cluster DF value for a term t in a cluster C is the percentage of docs in cluster C in which term t appears (so, if a cluster has 500 documents, and term "game" appears in 100 of those 500 documents, then DF value of "game" in that cluster is 0.2 or 20%). Sort the terms for each cluster in decreasing order of the DF percentage. Here is an example of how this output might look like (here the top 10 terms for 3 of the 5 clusters are displayed in decreasing order of cluster DF values, but the mean frequency from the cluster centroid is also shown). [Extra Credit: use your favorite third party tool, ideally with a Python based API, to create a word cloud for each cluster.]__

In [17]:
TD_tfidf = np.array(train)
centroids_tfidf, clusters_tfidf = kMeans(TD_tfidf, 5, distCosine, randCent)

In [18]:
clusters_tfidf.shape

(2000, 2)

In [19]:
centroids_tfidf.shape

(5, 9328)

In [20]:
i = centroids_tfidf.shape[1]
centroild2=[]
for i in range(i):
    centroild2.append(max(centroids_tfidf.T[i]))
centroid2=np.array(centroild2)

    

In [21]:
centroid=pd.DataFrame(centroid2)
centroid.columns=['Centroid']
centroid.head()

,Centroid
0,0.042942
1,0.030270
2,0.391884
3,0.056472
4,0.948409


In [22]:
clusters_tfidf

clusterdf=pd.DataFrame(clusters_tfidf)
clusterdf.columns = ['Cluster', 'X']
clusterdf.head()

,Cluster,X
0,2.0,0.780975
1,0.0,0.901787
2,2.0,0.725461
3,2.0,0.749828
4,3.0,0.813450


In [23]:
#mapping terms to documentFrequency
termsdf=pd.DataFrame(terms)
termsdf.columns = ['terms']
DFdf=pd.DataFrame(DF.T)
pNDocs=(DFdf/9328)*100
DFdf.columns = ['DF']
pNDocs.columns = ['%NDocs']

#newbie = pd.concat(termsdf,DFdf)

RESULTS=pd.concat([termsdf,DFdf,centroid,clusterdf['Cluster'], pNDocs ], axis=1)
#newbie.columns=['terms', 'TermFreqPerDoc']
RESULTS.head()



,terms,DF,Centroid,Cluster,%NDocs
0,aa,10,0.042942,2.0,0.107204
1,aargh,6,0.030270,0.0,0.064322
2,aaron,22,0.391884,2.0,0.235849
3,aaronc,9,0.056472,2.0,0.096484
4,ab,13,0.948409,3.0,0.139365


In [24]:
counts=pd.DataFrame((RESULTS['Cluster'].value_counts() ))
counts.columns=['NDocsPerCluster']
counts.index.name = 'CLUSTERS'
counts.index.astype=int
counts=counts.sort_values(by='CLUSTERS')
counts
#counts.iloc[ 5,: ]

,NDocsPerCluster
CLUSTERS,
0.0,469
1.0,2
2.0,371
3.0,1150
4.0,8


In [25]:
RESULTS[RESULTS['Cluster']==0].sort_values(by=['%NDocs','Cluster'], ascending=False).head(10)

,terms,DF,Centroid,Cluster,%NDocs
563,back,268,1.610812,0.0,2.873070
1317,chip,238,1.194889,0.0,2.551458
1567,comput,235,1.709234,0.0,2.519297
791,bit,175,92.076030,0.0,1.876072
1384,claim,153,0.599292,0.0,1.640223
756,bibl,139,0.790254,0.0,1.490137
357,applic,130,2.132672,0.0,1.393654
138,ago,127,0.317754,0.0,1.361492
1285,check,127,0.476651,0.0,1.361492
336,anywai,126,0.277367,0.0,1.350772


In [26]:
# Write a function to display the top N terms in each cluster along with the cluster
# DF values for each term and the size of the cluster. 
def TopNTerms(TD_tfidf, terms, DF, k ):
    centroids_tfidf, clusters_tfidf = kMeans(TD_tfidf, k, distCosine, randCent)
    termsdf=pd.DataFrame(terms)
    termsdf.columns = ['terms']
    DFdf=pd.DataFrame(DF.T)
    pNDocs=(DFdf/9328)*100
    DFdf.columns = ['DF']
    pNDocs.columns = ['%NDocs']
    RESULTS=pd.concat([termsdf,DFdf,centroid,clusterdf['Cluster'], pNDocs ], axis=1)
    counts=pd.DataFrame((RESULTS['Cluster'].value_counts() ))
    counts.columns=['NDocsPerCluster']
    counts.index.name = 'CLUSTERS'
    counts=counts.sort_values(by='CLUSTERS')
    n = centroids_tfidf.shape[0]
    
    for i in range(n):
       print("\nCluster:",i)
       print("\n\nCluster size:", counts.iloc[ i,: ] )
       print(RESULTS[RESULTS['Cluster']==i].sort_values(by=['%NDocs','Cluster'], ascending=False ).head(10))
       


In [27]:
TopNTerms(TD_tfidf, terms, DF, 5)


Cluster: 0


Cluster size: NDocsPerCluster    469
Name: 0.0, dtype: int64
       terms   DF   Centroid  Cluster    %NDocs
563     back  268   1.610812      0.0  2.873070
1317    chip  238   1.194889      0.0  2.551458
1567  comput  235   1.709234      0.0  2.519297
791      bit  175  92.076030      0.0  1.876072
1384   claim  153   0.599292      0.0  1.640223
756     bibl  139   0.790254      0.0  1.490137
357   applic  130   2.132672      0.0  1.393654
138      ago  127   0.317754      0.0  1.361492
1285   check  127   0.476651      0.0  1.361492
336   anywai  126   0.277367      0.0  1.350772

Cluster: 1


Cluster size: NDocsPerCluster    2
Name: 1.0, dtype: int64
          terms  DF  Centroid  Cluster    %NDocs
316      annual  16  0.621553      1.0  0.171527
1515  commision   3  0.025312      1.0  0.032161

Cluster: 2


Cluster size: NDocsPerCluster    371
Name: 2.0, dtype: int64
       terms   DF   Centroid  Cluster    %NDocs
417   articl  888   0.895978      2.0  9.519726
1084  

__d. Using the cluster assignments from Kmeans clustering, compare your 5 clusters to the 5 pre-assigned classes by computing the Completeness and Homogeneity values.__

In [28]:
from sklearn.metrics import completeness_score, homogeneity_score

print("Completeness Score:         ",completeness_score(target_train,ravel(clusters_tfidf.T[0])))
print("Homogeneity:                ",homogeneity_score(target_train,ravel(clusters_tfidf.T[0])))

Completeness Score:          0.7156123899771298
Homogeneity:                 0.4445973877194074


__e. Finally, using your cluster assignments as class labels, categorize each of the documents in the 20% set-aside data into each of the appropriate cluster. Your categorization should be based on Cosine similarity between each test document and cluster centroids. For each test document show the predicted class label as well as Cosine similarity to the corresponding cluster.__

In [29]:
#20% test data set aside
TD_tfidf_test = np.array(test)
centroids_tfidf_test, clusters_tfidf_test = kMeans(TD_tfidf_test, 4, distCosine, randCent)

In [30]:
print(TD_tfidf_test.shape,centroids_tfidf_test.shape)

(500, 9328) (4, 9328)


In [31]:
centroids_tfidf_test.T[9327]

array([ 0.21904982,  0.12944547,  0.        , 16.63456858])

In [32]:
1-distCosine(TD_tfidf[0],centroids_tfidf[0])

0.023683557321118043

In [33]:
TD_tfidf.shape

(2000, 9328)

In [43]:
result = []
for document in TD_tfidf_test:
    dicT = {}
    similarity = []
    count=1
    for centroid in centroids_tfidf_test:

        similarity.append(1-distCosine(document, centroid)) 
        dicT['SCluster '+ str(count)] = 1 - distCosine(document, centroid)
        count+=1
    dicT['PredictedCluster'] = similarity.index(max(similarity))+1
    result.append(dicT)

RESULTS = pd.DataFrame(result, index=range(len(TD_tfidf_test)))
RESULTS.columns=['PredictedCluster','SCluster 1','SCluster 2', 'SCluster 3','SCluster 4']
print(RESULTS['PredictedCluster'].value_counts())
RESULTS.head()

2    287
1    212
3      1
Name: PredictedCluster, dtype: int64


,PredictedCluster,SCluster 1,SCluster 2,SCluster 3,SCluster 4
0,1,0.099109,0.034571,0.000014,0.001551
1,2,0.119444,0.194061,0.000003,0.007016
2,2,0.021460,0.063866,0.000063,0.001053
3,2,0.105314,0.157973,0.000035,0.002178
4,2,0.070196,0.092188,0.000082,0.002623
